# Peer-graded Assignment
### Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np
import pandas as pd
import urllib.request

## 1: Scraping the Wikipedia table
### Function: souptable_to_dataframe()
Can be used for parsing an html-table contained in a beautiful soup bs.element.tag object.  
At least intended to be generic.

In [2]:
import bs4
from bs4 import BeautifulSoup

def souptable_to_dataframe(souptable):

    if not isinstance(souptable, bs4.element.Tag):
        print("souptable_to_dataframe: Wrong param type!")
        return pd.DataFrame()
    
    rows = souptable.find_all("tr")
    
    if len(rows) < 2:
        print("souptable_to_dataframe: No Table found!")
        return pd.DataFrame()
    
    # First the num of columns and their headers need to be determined 
    
    colcount = 0
    
    headers = rows[0].find_all('th')
    colcount = len(headers)
    
    columns = []
    
    if colcount > 0:
        for header in headers:
            columns.append(header.find(text=True).replace('\n',''))
        #print("Headers: ", ', '.join(columns))
        
    # when there are no headers present the first row is used to count the columns
    # and numeric headers are used
    
    else:        
        print("souptable_to_dataframe: No Header found, using first row for determining no. of cols")
        headers = rows[0].find_all('tr')
        colcount = len(headers)
        columns = range(0,colcount)

    # A 1D Array is filled with the cell values and reshaped later     
    
    vals = []
    
    rcnt = 0
    print(colcount)
    for row in souptable.find_all("tr"):
        ccnt = 0
        cols = row.find_all("td")
        if len(cols) >= colcount:
            rcnt = rcnt+1 
            for col in cols:
                if ccnt < colcount:
                    vals.append(col.find(text=True).replace('\n',''))
                    ccnt = ccnt+1
                    
    # Use a dict to create the dataframe with the column as keys 
    # For reshaping and transposing the array has to be transformed to an numpy matrix and later back to a list
    
    return pd.DataFrame(dict(zip(columns,np.array(vals).reshape(rcnt,ccnt).transpose().tolist())))

### Url retrieval
page content is read into a beatiful soup object

In [3]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
wikisoup = BeautifulSoup(urllib.request.urlopen(wikiurl), 'lxml')

### Table scraping and processing
Since there's only one table on the page find_all() is used to retrieve it.  
It's passed into the souptable_to_dataframe() function and all 'Not assigned' boroughs are removed from the resulting dataframe

In [4]:
torontotable=wikisoup.find_all('table', class_='wikitable sortable')
df_tor = souptable_to_dataframe(torontotable[0])
df_tor.head()

3


#### remove all boroughs which are 'Not assigned' from the dataframe 

In [5]:
df_tor = df_tor[df_tor['Borough'] != 'Not assigned']
df_tor.head()

#### replace 'Not assigned' neighborhoods with name of borough
**(!)** Only necessary for completing the task **(!)** since table currently has no rows where only neighborhood is not assigned 

In [19]:
for i,row in df_tor.loc[df_tor['Neighbourhood'] == "Not assigned",:].iterrows():
    df_tor.loc[i,'Neighbourhood'] = row['Borough']

df_tor.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Group by postcode and join Neighbourhoods with ','
Borough is the same for each group and 'max' selects one 

In [7]:
df_tor = df_tor.groupby('Postcode').agg({'Borough': 'max', 'Neighbourhood': lambda c: ",".join(c)}).reset_index()
df_tor.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_tor.shape

(103, 3)

# 2: Retrieving geospatial Data
### A try using pgeocode
As you can see I tried geocoder as well but didn't get any results!

In [9]:
#!pip install geocoder
!pip install pgeocode

In [10]:
import pgeocode

nom = pgeocode.Nominatim('ca')

df_ll = pd.DataFrame(nom.query_postal_code('XXX')).transpose()
df_ll = df_ll.drop(0)

for pcode in df_tor['Postcode']:
        df_ll = df_ll.append(nom.query_postal_code(pcode))

df_ll.head()

,postal_code,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,M1B,CA,Scarborough (Malvern / Rouge River),Ontario,ON,Scarborough,NaN,NaN,NaN,43.8113,-79.1930,6.0
0,M1C,CA,Scarborough (Rouge Hill / Port Union / Highlan...,Ontario,ON,Scarborough,NaN,NaN,NaN,43.7878,-79.1564,6.0
0,M1E,CA,Scarborough (Guildwood / Morningside / Ellesmere),Ontario,ON,Scarborough,NaN,NaN,NaN,43.7678,-79.1866,6.0
0,M1G,CA,Scarborough (Woburn),Ontario,ON,Scarborough,NaN,NaN,NaN,43.7712,-79.2144,6.0
0,M1H,CA,Scarborough (Cedarbrae),Ontario,ON,Scarborough,NaN,NaN,NaN,43.7686,-79.2389,6.0


In [11]:
df_ll.rename(columns={'postal_code': 'Postcode'}, inplace=True)

In [12]:
df_tor_fin = pd.merge(df_tor, df_ll[['Postcode', 'longitude','latitude']], on='Postcode')
df_tor_fin.head()

,Postcode,Borough,Neighbourhood,longitude,latitude
0,M1B,Scarborough,"Rouge,Malvern",-79.1930,43.8113
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",-79.1564,43.7878
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",-79.1866,43.7678
3,M1G,Scarborough,Woburn,-79.2144,43.7712
4,M1H,Scarborough,Cedarbrae,-79.2389,43.7686


### Location Data is missing for Post Gateway :(

In [13]:
df_tor_fin[df_tor_fin['longitude'].isna()]

,Postcode,Borough,Neighbourhood,longitude,latitude
86,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


### Different approach using an API:

In [14]:
import requests

ll_csv = open("ll.csv","w")
ll_csv.write("dunno,dunno2,lat,long\n")

url = "https://geocoder.ca/"

# Don't wanna waste too much time here
cnt = 0

for pcode in df_tor['Postcode']:
    address = 'Toronto,{}1x9,Canada'.format(pcode)
    params = {'locate': address, 'geoit':"csv"}

    result = requests.get(url,params=params)

    ll_csv.write(result.content.decode("utf-8")+'\n')
    
    cnt=cnt+1
    if cnt > 4: 
        break

ll_csv.close()

<h3> The API seems to have only very limited daily requests with a free account :(</h3>
<p>If you call it from your browser you actually get 
    <a href="https://geocoder.ca/?locate=Toronto,M1B1x9,Canada&geoit=csv">results</a>. My notebook's are already used up:</p>

In [15]:
df_ll = pd.read_csv("ll.csv")
df_ll

,dunno,dunno2,lat,long
0,6,Throttled,NaN,NaN
1,6,Throttled,NaN,NaN
2,6,Throttled,NaN,NaN
3,6,Throttled,NaN,NaN
4,6,Throttled,NaN,NaN


### So just take the file from the assignment
Below here is a hidden cell containing Watson Studio file importing code with credentials  
The resulting DataFrame is **df_data_1**

In [16]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


 ### Merge the two dataframes

In [17]:
df_tor_fin = pd.merge(df_tor.rename(columns={'Postcode': 'Postal Code'}), df_data_1, on="Postal Code")
df_tor_fin.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### No missing data :)

In [18]:
df_tor_fin[df_tor_fin['Longitude'].isna()]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
